In [1]:
import argparse
import pandas as pd
from tqdm import tqdm
import yaml
import math
import numpy as np
from datetime import datetime
import json
import bisect
from tqdm import tqdm

config = yaml.load(open('../../config/config.yaml'), Loader=yaml.FullLoader)


In [2]:
nc_down = pd.read_csv('../../'+config['data']['raw']['train']['nc_down'])
nc_sample_label = pd.read_csv('../../' + config['data']['raw']['train']['nc_sample_label'])

In [9]:
nc_down

,nc_ip,down_start_time,is_active
0,f1e10ce9-f8f9-4114-8fb0-c83a52947844,2020-03-10 16:27:20,True
1,e3b9909a-d9a0-4188-ad30-1d131ccb60dc,2019-11-15 09:25:24,True
2,b539e341-4a45-4001-9aa1-f6a44f996552,2016-12-18 12:38:12,True
3,9e0d0a75-5796-41e5-845b-558f3ef32685,2019-08-08 02:44:26,True
4,bcb19ebd-58b6-4114-b266-2cd3df8c7867,2020-01-07 13:00:40,True
...,...,...,...
201,9988b887-4fe9-48de-b207-eab2407100f8,2020-03-29 17:01:42,True
202,ab535065-0b03-4368-8065-569fdd9dc2e2,2018-04-19 06:04:47,False
203,4ec32f4e-bd98-4865-8da2-0a06fdc6c350,2020-10-02 12:10:08,True
204,7ec737a3-ae07-4db3-a689-90dff57010bf,2019-07-31 21:14:50,True


In [6]:
# Pre declare some date-util functions
print('Pre declare some date-util functions')
str2date = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f") if '.' in x else datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
series2date = lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S')
date2str = lambda x: x.strftime("%Y-%m-%d %H:%M:%S.%f")


Pre declare some date-util functions


In [10]:
res_tuples = []
res_columns = ['nc_ip', 'sample_time', 'nc_down_label']
for i in tqdm(range(nc_down.shape[0]), total=nc_down.shape[0]):
    down_time = str2date(nc_down.loc[i, 'down_start_time'])
    nc_ip = nc_down.loc[i, 'nc_ip']
    for i in range(0, 60*24*2, 5):
        sample_time = down_time - pd.DateOffset(minutes=i)
        res_tuples.append(
            (nc_ip, sample_time, True)
        )
    for i in range(60*24*2, 60*24*7, 5):
        sample_time = down_time - pd.DateOffset(minutes=i)
        res_tuples.append(
            (nc_ip, sample_time, False)
        )



100%|██████████| 206/206 [00:06<00:00, 31.57it/s]


In [11]:
res_df = pd.DataFrame(res_tuples, columns=res_columns)

In [16]:
res_df

,nc_ip,sample_time,nc_down_label
0,f1e10ce9-f8f9-4114-8fb0-c83a52947844,2020-03-10 16:27:20,True
1,f1e10ce9-f8f9-4114-8fb0-c83a52947844,2020-03-10 16:22:20,True
2,f1e10ce9-f8f9-4114-8fb0-c83a52947844,2020-03-10 16:17:20,True
3,f1e10ce9-f8f9-4114-8fb0-c83a52947844,2020-03-10 16:12:20,True
4,f1e10ce9-f8f9-4114-8fb0-c83a52947844,2020-03-10 16:07:20,True
...,...,...,...
415291,d33be639-de4b-4865-9fe2-d8c7ee171a2d,2018-07-11 03:13:45,False
415292,d33be639-de4b-4865-9fe2-d8c7ee171a2d,2018-07-11 03:08:45,False
415293,d33be639-de4b-4865-9fe2-d8c7ee171a2d,2018-07-11 03:03:45,False
415294,d33be639-de4b-4865-9fe2-d8c7ee171a2d,2018-07-11 02:58:45,False


In [15]:
res_df.to_csv('../../'+config['data']['process']['aug_train']['nc_sample_label'], index=False)